In [2]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Flatten,Embedding,Dense
from tensorflow.keras.models import Sequential
import numpy as np

# I Introduction au problème

On s'inspire de ce [tuto](https://medium.com/analytics-vidhya/understanding-embedding-layer-in-keras-bbe3ff1327ce#:~:text=Embedding%20layer%20enables%20us%20to,way%20along%20with%20reduced%20dimensions)
L'idée est de classifier des avis en fonction qu'ils sont positifs (1) ou  négatif (0)

In [20]:
# Define 10 restaurant reviews
reviews =[
          'Never coming back!, Never!',
          'horrible service',
          'rude waitress',
          'cold food',
          'horrible food!',
          'awesome, food is okay, atmosphere great',
          'awesome services!, I love this place',
          'rocks',
          'poor work, my eyes crie',
          'couldn\'t have done better, waitress is super nice'
]
#Define labels
labels = array([0,0,0,0,0,1,1,1,0,1])

# Entrainer son propre embedding

In [24]:
Vocab_size = 50
encoded_reviews = [one_hot(d,Vocab_size) for d in reviews]
print(f'encoded reviews:')
for t in encoded_reviews:
    print (t)

encoded reviews:
[3, 20, 22, 3]
[28, 36]
[20, 26]
[42, 13]
[28, 13]
[27, 13, 1, 9, 44, 2]
[27, 24, 48, 3, 23, 17]
[26]
[9, 15, 9, 13, 14]
[10, 15, 15, 42, 26, 1, 17, 13]


In [22]:
max_length = 5
padded_reviews = pad_sequences(encoded_reviews,maxlen=max_length,padding='post')
print(f'paded reviews:')
for t in padded_reviews:
    print (t)

paded reviews:
[ 3 20 22  3  0]
[28 36  0  0  0]
[20 26  0  0  0]
[42 13  0  0  0]
[28 13  0  0  0]
[13  1  9 44  2]
[24 48  3 23 17]
[26  0  0  0  0]
[ 9 15  9 13 14]
[42 26  1 17 13]


In [11]:
model = Sequential()
embedding_layer = Embedding(input_dim=Vocab_size,output_dim=8,input_length=max_length)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
print(model.summary())

2022-05-04 10:51:18.716190: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 8)              400       
                                                                 
 flatten (Flatten)           (None, 40)                0         
                                                                 
 dense (Dense)               (None, 1)                 41        
                                                                 
Total params: 441
Trainable params: 441
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
model.fit(padded_reviews,labels,epochs=100,verbose=0)

In [13]:
print(embedding_layer.get_weights()[0].shape)

(50, 8)


In [25]:
print(embedding_layer.get_weights()[0][2])

[-0.03984213 -0.06080137  0.13901348  0.06346074 -0.05954554  0.02771615
  0.14004043 -0.09462351]


# III Utiliser un embedding pré-entrainé (Glove)

In [16]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
df_train = pd.DataFrame(data= np.c_[twenty_train['data'], twenty_train['target']], columns= ['text','target'])
df_test = pd.DataFrame(data= np.c_[twenty_test['data'], twenty_test['target']], columns= ['text','target'])

In [19]:
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
vectorizer.adapt(df_train)

ValueError: in user code:

    File "/Users/charles/Documents/pythonProject/NLP_introduction/env/lib/python3.9/site-packages/keras/engine/base_preprocessing_layer.py", line 118, in adapt_step  *
        self.update_state(data)
    File "/Users/charles/Documents/pythonProject/NLP_introduction/env/lib/python3.9/site-packages/keras/layers/preprocessing/text_vectorization.py", line 431, in update_state  **
        self._lookup_layer.update_state(self._preprocess(data))
    File "/Users/charles/Documents/pythonProject/NLP_introduction/env/lib/python3.9/site-packages/keras/layers/preprocessing/text_vectorization.py", line 520, in _preprocess
        raise ValueError(

    ValueError: When using `TextVectorization` to tokenize strings, the input rank must be 1 or the last shape dimension must be 1. Received: inputs.shape=(None, 2) with rank=2
